![title](banner.png)

In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('Data.csv')
data["img"] = "mac.png"

In [2]:
from bokeh.io import show, output_notebook, push_notebook
from bokeh.plotting import figure
from bokeh.models import HoverTool, ColumnDataSource, ColorBar, LinearColorMapper, ImageURL, BoxAnnotation, Label
from bokeh.layouts import Row, Column
from bokeh.palettes import RdYlGn
from ipywidgets import interact, IntSlider, IntText, Dropdown

In [3]:
%%HTML
<div class="flourish-embed" data-src="visualisation/52730"></div><script src="https://public.flourish.studio/resources/embed.js"></script>

* *Size of dot represents the amount of McDonald's restaurants per 1M inhabitants of a country*

In [4]:
hover = HoverTool(tooltips=[("Country", "@Country_Name"),("Population", "@population"),("McDonald's locations", "@mcDonaldsAmount")])
mcdonalds_per_country = data[data["mcDonaldsPerPopulation"] > 0]
source = ColumnDataSource(mcdonalds_per_country.groupby("Country_Name").mean())

output_notebook()

p1 = figure(title="Which countries have the highest McDonald's density?",y_axis_label="McDonald's locations per 1M population",x_range=mcdonalds_per_country.sort_values("mcDonaldsPerPopulation", ascending=False)["Country_Name"].unique(), y_range=(0,55), plot_width=2000, plot_height=400, outline_line_color=None,tools=[hover])
s1 = p1.vbar(x="Country_Name", top="mcDonaldsPerPopulation", color="mediumseagreen",hover_color="green", width=0.5, source=source)
p1.xaxis.major_label_orientation = "vertical"

show(p1, notebook_handle=True)

def update(f):
    s1.data_source.data = ColumnDataSource.from_df(mcdonalds_per_country[mcdonalds_per_country["year"] == f])
    push_notebook()

menu = Dropdown(options=[2010, 2015],value=2010,description='Year:')
interact(update, f=menu)

Loading BokehJS ...

interactive(children=(Dropdown(description='Year:', options=(2010, 2015), value=2010), Output()), _dom_classes=('widget-interact',))

<function __main__.update>

![title](banner1.png)

In [6]:
hover = HoverTool(tooltips=[("Country", "@Country_Name"),("Big Mac price", "@bigMacPrice $"),("Average yearly income", "@gniPerCap $"),("Average yearly working time", "@averageWorkingHours hrs."),("Working time to buy one Big Mac", "@hoursPerBigMac hrs.")],names=["circle"])
tools = ["box_select", hover, "reset"]

data["radius_price"] = (((data["bigMacPrice"] - data["bigMacPrice"].min())/(data["bigMacPrice"] - data["bigMacPrice"].min()).max())*4500) + 500
data["radius_wage"] = (((data["wagePerHour"] - data["wagePerHour"].min())/(data["wagePerHour"] - data["wagePerHour"].min()).max())*70) + 5
source = ColumnDataSource(data[data["year"]==2000])

p2 = figure(title="Working harder for the same product",x_axis_label='Mean yearly income per capita ($)', y_axis_label='Hours of work needed for one Big Mac', width=480, height=450, x_range=(-2000,120000), y_range=(0,3), tools=tools)
s2 = p2.scatter("gniPerCap", "hoursPerBigMac", source=source,radius="radius_price",hover_color="lightcoral",fill_alpha=0.3,color="lightcoral",name="circle")

p3 = figure(title="Labor inequality has grown significantly",x_axis_label='Yearly working time per capita (hrs.)',y_axis_label='Mean hourly income per capita ($)',x_range=(1300,2500),y_range=(-1,80),width=480, height=450, tools=tools)
s3 = p3.scatter("averageWorkingHours", "wagePerHour", source=source,radius="radius_wage",hover_color="cornflowerblue",fill_alpha=0.2,color="cornflowerblue",name="circle")   

for x in data["Country_Name"].unique():
    p2.line("gniPerCap","hoursPerBigMac",source=ColumnDataSource(data[data["Country_Name"]==x].sort_values("year")),line_alpha=0.15,level='underlay',color="gray")
    
def update(f):
    s2.data_source.data = ColumnDataSource.from_df(data[data["year"] == f])
    push_notebook()

show(Row(p2,p3), notebook_handle=True)

menu = IntSlider(min=2000,max=2016,step=1,value=2000,description="Year:")
interact(update, f=menu)

interactive(children=(IntSlider(value=2000, description='Year:', max=2016, min=2000), Output()), _dom_classes=('widget-interact',))

<function __main__.update>

![title](banner2.png)

In [7]:
hover = HoverTool(tooltips=[("Country", "@Country_Name"),("Population", "@population"),("McDonald's locations", "@mcDonaldsAmount"), ("Average yearly income", "@gniPerCap $")])
tools = ["box_select", hover, "reset"]
source = ColumnDataSource(data[data["year"]==2010])

p4 = figure(title="McDonald's does not grow linearly with population",x_axis_label='Total population', y_axis_label="Amount of McDonald's locations", x_axis_type="log", y_axis_type="log",x_range=[5e4, 5e9],y_range=[10e-1, 10e4],width=480, height=450, tools=tools)
s4 = p4.scatter("population", "mcDonaldsAmount",source=source,fill_alpha=0.2,color="purple",hover_color="purple")

p5 = figure(title="The richer a country, the more people have access to McDonald's",x_axis_label='Mean yearly income per capita ($)', y_axis_label="McDonald's locations per 1M population",x_range=(0,99000),y_range=(-1,50),width=480, height=450, tools=tools)
s5 = p5.scatter("gniPerCap", "mcDonaldsPerPopulation",radius=2300, source=source,color="lightcoral",hover_color="red")
p5.image_url(url="img",w=3800,h=1.8,x="gniPerCap",y="mcDonaldsPerPopulation",anchor="center",global_alpha=0.9,source=source)

show(Row(p4,p5), notebook_handle=True)

def update(f):
    s4.data_source.data = ColumnDataSource.from_df(mcdonalds_per_country[mcdonalds_per_country["year"] == f])
    push_notebook()

menu = Dropdown(options=[2010, 2015],value=2010,description='Year:')
interact(update, f=menu)

interactive(children=(Dropdown(description='Year:', options=(2010, 2015), value=2010), Output()), _dom_classes=('widget-interact',))

<function __main__.update>

![title](banner3.png)

In [8]:
hover = HoverTool(tooltips=[("Year", "@year"),("Big Macs per hour", "@bigMacPerHour{int}"), ("Calories per hour", "@caloriesPerHour{int}")])
cal_by_country = ColumnDataSource(data[data["bigMacPerHour"] >= 0].groupby("Country_Name").mean())
cal_by_year = ColumnDataSource(data.groupby("year").mean())

p6 = figure(title="How many calories can one hour of work buy you?",x_axis_label='Year',y_axis_label='Food affordance (cal.)',plot_width=900, plot_height=250,y_range=(1000,3200),tools=[hover])
s6 = p6.line(x="year", y="caloriesPerHour",line_width=2, color="gray",source=cal_by_year)

hover = HoverTool(tooltips=[("Country", "@Country_Name"),("Big Macs per hour", "@bigMacPerHour{int}"), ("Calories per hour", "@caloriesPerHour{int}")])
p7 = figure(y_axis_label='Food affordance (cal.)',plot_width=900, plot_height=350,x_range=data[data["bigMacPerHour"] >= 0].groupby("Country_Name"),y_range=(0,5300),tools=[hover])
box_bottom = BoxAnnotation(top=2000, fill_alpha=0.1, fill_color='red',level='underlay',line_color="black",line_alpha=1,line_dash="dashed")
box_top = BoxAnnotation(bottom=2000, fill_alpha=0.1, fill_color='green',level='underlay')

p6.renderers.extend([box_bottom, box_top])
p7.renderers.extend([box_bottom, box_top])

s7 = p7.vbar(x="Country_Name", top="caloriesPerHour", color="darkgray",hover_color="gray", width=0.6, source=cal_by_country)
p7.xaxis.major_label_orientation = "vertical"

show(Column(p6,p7), notebook_handle=True)

def update(a,b,c,d):
    if (a == "Male") :
        box_bottom.top = box_top.bottom = (66.5 + 13.8*c + 5*d - 6.8*b)*1.3
    else :
        box_bottom.top = box_top.bottom = (655.1 + 9.563*c + 1.850*d - 4.676*b)*1.3
    
    push_notebook()

gender = Dropdown(options=["Male", "Female"],description='Gender:')
age = IntText(value=20,description='Age:')
weight = IntText(value=70,description='Weight (kg):')
height = IntText(value=180,description='Height (cm):')
interact(update, a=gender, b=age, c=weight, d=height,continuous_update=False)

interactive(children=(Dropdown(description='Gender:', options=('Male', 'Female'), value='Male'), IntText(value=20, description='Age:'), IntText(value=70, description='Weight (kg):'), IntText(value=180, description='Height (cm):'), Output()), _dom_classes=('widget-interact',))

<function __main__.update>